### Scrape Booking.com 

Since BookingHoldings doesn't have aggregated databases, it will be much faster to scrape data directly from booking.com 

You can scrap as many information asyou want, but we suggest that you get at least:

*   hotel name,
*   Url to its booking.com page,
*   Its coordinates: latitude and longitude
*   Score given by the website users
*   Text description of the hotel

# 1) Création d'un DataFrame ville/URL

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm import tqdm
import datetime as dt

In [2]:
cities =["Mont-Saint-Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]

Nous faisons les changements suivant sur les recherches booking car les villes en question ne permettent pas d'accéder aux balises recherchées

Chateau du Haut Koenigsbourg -> Orschwiller  <br />
Gorges du Verdon -> moustiers sainte marie  <br />
Ariege -> Ax-les-Thermes

In [3]:
cities_url =['https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB&sid=2d385a5c8721c18a0aca0b4388353818&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB%3Bsid%3D2d385a5c8721c18a0aca0b4388353818%3Bsb_price_type%3Dtotal%26%3B&ss=Le+Mont-Saint-Michel%2C+Basse-Normandie%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=Mont-Saint-Michel&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=900039327&dest_type=city&place_id_lat=48.636&place_id_lon=-1.511&search_pageview_id=f07e69615ba901c1&search_selected=true',
             'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB&sid=2d385a5c8721c18a0aca0b4388353818&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB%3Bsid%3D2d385a5c8721c18a0aca0b4388353818%3Bsb_price_type%3Dtotal%26%3B&ss=Saint-Malo%2C+Bretagne%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=St+Malo&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1466824&dest_type=city&place_id_lat=48.649616&place_id_lon=-2.025677&search_pageview_id=f07e69615ba901c1&search_selected=true&search_pageview_id=f07e69615ba901c1&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0',
             'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB&sid=2d385a5c8721c18a0aca0b4388353818&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB%3Bsid%3D2d385a5c8721c18a0aca0b4388353818%3Bsb_price_type%3Dtotal%26%3B&ss=Bayeux%2C+Basse-Normandie%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=Bayeux&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1410836&dest_type=city&place_id_lat=49.27605&place_id_lon=-0.70229&search_pageview_id=f07e69615ba901c1&search_selected=true',
             'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB&sid=2d385a5c8721c18a0aca0b4388353818&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB%3Bsid%3D2d385a5c8721c18a0aca0b4388353818%3Bsb_price_type%3Dtotal%26%3B&ss=Le+Havre%2C+Haute-Normandie%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=Le+Havre&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1441598&dest_type=city&place_id_lat=49.49489&place_id_lon=0.107723&search_pageview_id=f07e69615ba901c1&search_selected=true',
             'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB&sid=2d385a5c8721c18a0aca0b4388353818&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB%3Bsid%3D2d385a5c8721c18a0aca0b4388353818%3Bsb_price_type%3Dtotal%26%3B&ss=Rouen%2C+Haute-Normandie%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=Rouen&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1462807&dest_type=city&place_id_lat=49.44046&place_id_lon=1.09397&search_pageview_id=f07e69615ba901c1&search_selected=true',
             'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB&sid=2d385a5c8721c18a0aca0b4388353818&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB%3Bsid%3D2d385a5c8721c18a0aca0b4388353818%3Bsb_price_type%3Dtotal%26%3B&ss=Paris%2C+%C3%8Ele-de-France%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=Paris&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1456928&dest_type=city&iata=PAR&place_id_lat=48.85668&place_id_lon=2.351476&search_pageview_id=f07e69615ba901c1&search_selected=true',
             'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB&sid=2d385a5c8721c18a0aca0b4388353818&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB%3Bsid%3D2d385a5c8721c18a0aca0b4388353818%3Bsb_price_type%3Dtotal%26%3B&ss=Amiens%2C+Picardie%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=Amiens&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1407447&dest_type=city&place_id_lat=49.895454&place_id_lon=2.299816&search_pageview_id=f07e69615ba901c1&search_selected=true',
             'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB&sid=2d385a5c8721c18a0aca0b4388353818&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB%3Bsid%3D2d385a5c8721c18a0aca0b4388353818%3Bsb_price_type%3Dtotal%26%3B&ss=Lille%2C+Nord-Pas-de-Calais%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=Lille&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1447079&dest_type=city&iata=LIL&place_id_lat=50.636845&place_id_lon=3.063454&search_pageview_id=f07e69615ba901c1&search_selected=true',
             'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB&sid=2d385a5c8721c18a0aca0b4388353818&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB%3Bsid%3D2d385a5c8721c18a0aca0b4388353818%3Bsb_price_type%3Dtotal%26%3B&ss=Strasbourg%2C+Alsace%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=Strasbourg&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1471697&dest_type=city&iata=SXB&place_id_lat=48.5834&place_id_lon=7.7458&search_pageview_id=f07e69615ba901c1&search_selected=true',
             'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AtW-x40GwAIB0gIkMDIyMWJkMzItMDdjNy00OGJhLWFhYmEtZWQxYzcyN2IzNWYx2AIE4AIB&sid=2d385a5c8721c18a0aca0b4388353818&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AtW-x40GwAIB0gIkMDIyMWJkMzItMDdjNy00OGJhLWFhYmEtZWQxYzcyN2IzNWYx2AIE4AIB%3Bsid%3D2d385a5c8721c18a0aca0b4388353818%3Bsb_price_type%3Dtotal%26%3B&ss=Orschwiller%2C+Alsace%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=Orschwiller+&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1456230&dest_type=city&place_id_lat=48.24224&place_id_lon=7.37998&search_pageview_id=24544c2a63dc00a2&search_selected=true&search_pageview_id=24544c2a63dc00a2&ac_suggestion_list_length=1&ac_suggestion_theme_list_length=0',
             'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AtW-x40GwAIB0gIkMDIyMWJkMzItMDdjNy00OGJhLWFhYmEtZWQxYzcyN2IzNWYx2AIE4AIB&sid=2d385a5c8721c18a0aca0b4388353818&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AtW-x40GwAIB0gIkMDIyMWJkMzItMDdjNy00OGJhLWFhYmEtZWQxYzcyN2IzNWYx2AIE4AIB%3Bsid%3D2d385a5c8721c18a0aca0b4388353818%3Bsb_price_type%3Dtotal%26%3B&ss=Colmar%2C+Alsace%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=Colmar&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1421049&dest_type=city&place_id_lat=48.079155&place_id_lon=7.357718&search_pageview_id=24544c2a63dc00a2&search_selected=true&search_pageview_id=24544c2a63dc00a2&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0',
             'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB&sid=2d385a5c8721c18a0aca0b4388353818&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB%3Bsid%3D2d385a5c8721c18a0aca0b4388353818%3Bsb_price_type%3Dtotal%26%3B&ss=Eguisheim%2C+Alsace%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&search_pageview_id=f07e69615ba901c1&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1425030&dest_type=city&place_id_lat=48.04231&place_id_lon=7.30543&search_pageview_id=f07e69615ba901c1&search_selected=true&ss_raw=Eguisheim',
             'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB&sid=2d385a5c8721c18a0aca0b4388353818&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB%3Bsid%3D2d385a5c8721c18a0aca0b4388353818%3Bsb_price_type%3Dtotal%26%3B&ss=Besan%C3%A7on%2C+Franche-Comt%C3%A9%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=Besancon&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1412198&dest_type=city&place_id_lat=47.240215&place_id_lon=6.023773&search_pageview_id=f07e69615ba901c1&search_selected=true',
             'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB&sid=2d385a5c8721c18a0aca0b4388353818&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB%3Bsid%3D2d385a5c8721c18a0aca0b4388353818%3Bsb_price_type%3Dtotal%26%3B&ss=Dijon%2C+Bourgogne%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=Dijon&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1423981&dest_type=city&place_id_lat=47.321762&place_id_lon=5.039809&search_pageview_id=f07e69615ba901c1&search_selected=true',
             'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AtW-x40GwAIB0gIkMDIyMWJkMzItMDdjNy00OGJhLWFhYmEtZWQxYzcyN2IzNWYx2AIE4AIB&sid=2d385a5c8721c18a0aca0b4388353818&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AtW-x40GwAIB0gIkMDIyMWJkMzItMDdjNy00OGJhLWFhYmEtZWQxYzcyN2IzNWYx2AIE4AIB%3Bsid%3D2d385a5c8721c18a0aca0b4388353818%3Bsb_price_type%3Dtotal%26%3B&ss=Annecy%2C+Rh%C3%B4ne-Alpes%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=Annecy&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1407760&dest_type=city&iata=NCY&place_id_lat=45.899727&place_id_lon=6.127196&search_pageview_id=24544c2a63dc00a2&search_selected=true&search_pageview_id=24544c2a63dc00a2&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0',
             'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB&sid=2d385a5c8721c18a0aca0b4388353818&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB%3Bsid%3D2d385a5c8721c18a0aca0b4388353818%3Bsb_price_type%3Dtotal%26%3B&ss=Grenoble%2C+Rh%C3%B4ne-Alpes%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=Grenoble&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1430647&dest_type=city&place_id_lat=45.192505&place_id_lon=5.724973&search_pageview_id=f07e69615ba901c1&search_selected=true',
             'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB&sid=2d385a5c8721c18a0aca0b4388353818&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB%3Bsid%3D2d385a5c8721c18a0aca0b4388353818%3Bsb_price_type%3Dtotal%26%3B&ss=Lyon%2C+Rh%C3%B4ne-Alpes%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=Lyon&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1448468&dest_type=city&iata=LYS&place_id_lat=45.757767&place_id_lon=4.832204&search_pageview_id=f07e69615ba901c1&search_selected=true',
             'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AtW-x40GwAIB0gIkMDIyMWJkMzItMDdjNy00OGJhLWFhYmEtZWQxYzcyN2IzNWYx2AIE4AIB&sid=2d385a5c8721c18a0aca0b4388353818&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AtW-x40GwAIB0gIkMDIyMWJkMzItMDdjNy00OGJhLWFhYmEtZWQxYzcyN2IzNWYx2AIE4AIB%3Bsid%3D2d385a5c8721c18a0aca0b4388353818%3Bsb_price_type%3Dtotal%26%3B&ss=Moustiers-Sainte-Marie%2C+Provence-Alpes-C%C3%B4te+d%27Azur%2C+France&is_ski_area=&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=Moustiers-Sainte-Marie&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1454083&dest_type=city&place_id_lat=43.847195&place_id_lon=6.222247&search_pageview_id=24544c2a63dc00a2&search_selected=true&search_pageview_id=24544c2a63dc00a2&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0',
             'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB&sid=2d385a5c8721c18a0aca0b4388353818&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB%3Bsid%3D2d385a5c8721c18a0aca0b4388353818%3Bsb_price_type%3Dtotal%26%3B&ss=Bormes-les-Mimosas%2C+Provence-Alpes-C%C3%B4te+d%27Azur%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=Bormes+les+Mimosas&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1413801&dest_type=city&place_id_lat=43.1507&place_id_lon=6.34193&search_pageview_id=f07e69615ba901c1&search_selected=true',
             'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB&sid=2d385a5c8721c18a0aca0b4388353818&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB%3Bsid%3D2d385a5c8721c18a0aca0b4388353818%3Bsb_price_type%3Dtotal%26%3B&ss=Cassis%2C+Provence-Alpes-C%C3%B4te+d%27Azur%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=Cassis&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1416912&dest_type=city&place_id_lat=43.21513&place_id_lon=5.53712&search_pageview_id=f07e69615ba901c1&search_selected=true',
             'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB&sid=2d385a5c8721c18a0aca0b4388353818&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB%3Bsid%3D2d385a5c8721c18a0aca0b4388353818%3Bsb_price_type%3Dtotal%26%3B&ss=Marseille%2C+Provence-Alpes-C%C3%B4te+d%27Azur%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=Marseille&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1449947&dest_type=city&iata=MRS&place_id_lat=43.2983&place_id_lon=5.37798&search_pageview_id=f07e69615ba901c1&search_selected=true',
             'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB&sid=2d385a5c8721c18a0aca0b4388353818&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB%3Bsid%3D2d385a5c8721c18a0aca0b4388353818%3Bsb_price_type%3Dtotal%26%3B&ss=Aix-en-Provence%2C+Provence-Alpes-C%C3%B4te+d%27Azur%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=Aix+en+Provence&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1406939&dest_type=city&place_id_lat=43.526314&place_id_lon=5.445464&search_pageview_id=f07e69615ba901c1&search_selected=true',
             'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB&sid=2d385a5c8721c18a0aca0b4388353818&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB%3Bsid%3D2d385a5c8721c18a0aca0b4388353818%3Bsb_price_type%3Dtotal%26%3B&ss=Avignon%2C+Provence-Alpes-C%C3%B4te+d%27Azur%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=Avignon&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1409631&dest_type=city&iata=AVN&place_id_lat=43.949&place_id_lon=4.807508&search_pageview_id=f07e69615ba901c1&search_selected=true',
             'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB&sid=2d385a5c8721c18a0aca0b4388353818&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB%3Bsid%3D2d385a5c8721c18a0aca0b4388353818%3Bsb_price_type%3Dtotal%26%3B&ss=Uz%C3%A8s%2C+Languedoc-Roussillon%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=Uzes&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1474231&dest_type=city&place_id_lat=44.01217&place_id_lon=4.41967&search_pageview_id=f07e69615ba901c1&search_selected=true&search_pageview_id=f07e69615ba901c1&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0',
             'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB&sid=2d385a5c8721c18a0aca0b4388353818&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB%3Bsid%3D2d385a5c8721c18a0aca0b4388353818%3Bsb_price_type%3Dtotal%26%3B&ss=N%C3%AEmes%2C+Languedoc-Roussillon%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=Nimes&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1455068&dest_type=city&iata=FNI&place_id_lat=43.83742&place_id_lon=4.36007&search_pageview_id=f07e69615ba901c1&search_selected=true',
             'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB&sid=2d385a5c8721c18a0aca0b4388353818&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB%3Bsid%3D2d385a5c8721c18a0aca0b4388353818%3Bsb_price_type%3Dtotal%26%3B&ss=Aigues-Mortes%2C+Languedoc-Roussillon%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=Aigues+Mortes&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1406800&dest_type=city&place_id_lat=43.566887&place_id_lon=4.190193&search_pageview_id=f07e69615ba901c1&search_selected=true',
             'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB&sid=2d385a5c8721c18a0aca0b4388353818&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB%3Bsid%3D2d385a5c8721c18a0aca0b4388353818%3Bsb_price_type%3Dtotal%26%3B&ss=Les+Saintes-Maries-de-la-Mer%2C+Provence-Alpes-C%C3%B4te+d%27Azur%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=Saintes+Maries+de+la+mer&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1465138&dest_type=city&place_id_lat=43.45233&place_id_lon=4.42871&search_pageview_id=f07e69615ba901c1&search_selected=true',
             'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB&sid=2d385a5c8721c18a0aca0b4388353818&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB%3Bsid%3D2d385a5c8721c18a0aca0b4388353818%3Bsb_price_type%3Dtotal%26%3B&ss=Collioure%2C+Languedoc-Roussillon%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=Collioure&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1421032&dest_type=city&place_id_lat=42.52453&place_id_lon=3.082255&search_pageview_id=f07e69615ba901c1&search_selected=true',
             'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB&sid=2d385a5c8721c18a0aca0b4388353818&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB%3Bsid%3D2d385a5c8721c18a0aca0b4388353818%3Bsb_price_type%3Dtotal%26%3B&ss=Carcassonne%2C+Languedoc-Roussillon%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=Carcassonne&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1416701&dest_type=city&iata=CCF&place_id_lat=43.21304&place_id_lon=2.34911&search_pageview_id=f07e69615ba901c1&search_selected=true',
             'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AtW-x40GwAIB0gIkMDIyMWJkMzItMDdjNy00OGJhLWFhYmEtZWQxYzcyN2IzNWYx2AIE4AIB&sid=2d385a5c8721c18a0aca0b4388353818&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AtW-x40GwAIB0gIkMDIyMWJkMzItMDdjNy00OGJhLWFhYmEtZWQxYzcyN2IzNWYx2AIE4AIB%3Bsid%3D2d385a5c8721c18a0aca0b4388353818%3Bsb_price_type%3Dtotal%26%3B&ss=Ax-les-Thermes%2C+Midi-Pyr%C3%A9n%C3%A9es%2C+France&is_ski_area=1&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=ax+les+&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1409718&dest_type=city&place_id_lat=42.71973&place_id_lon=1.83798&search_pageview_id=24544c2a63dc00a2&search_selected=true&search_pageview_id=24544c2a63dc00a2&ac_suggestion_list_length=5&ac_suggestion_theme_list_length=0',
             'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB&sid=2d385a5c8721c18a0aca0b4388353818&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB%3Bsid%3D2d385a5c8721c18a0aca0b4388353818%3Bsb_price_type%3Dtotal%26%3B&ss=Toulouse%2C+Midi-Pyr%C3%A9n%C3%A9es%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=Toulouse&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1473166&dest_type=city&iata=TLS&place_id_lat=43.604645&place_id_lon=1.444213&search_pageview_id=f07e69615ba901c1&search_selected=true',
             'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB&sid=2d385a5c8721c18a0aca0b4388353818&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB%3Bsid%3D2d385a5c8721c18a0aca0b4388353818%3Bsb_price_type%3Dtotal%26%3B&ss=Montauban%2C+Midi-Pyr%C3%A9n%C3%A9es%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=Montauban&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1452421&dest_type=city&place_id_lat=44.01758&place_id_lon=1.355&search_pageview_id=f07e69615ba901c1&search_selected=true',
             'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB&sid=2d385a5c8721c18a0aca0b4388353818&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB%3Bsid%3D2d385a5c8721c18a0aca0b4388353818%3Bsb_price_type%3Dtotal%26%3B&ss=Biarritz%2C+Aquitaine%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=Biarritz&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1412526&dest_type=city&iata=BIQ&place_id_lat=43.4804&place_id_lon=-1.56066&search_pageview_id=f07e69615ba901c1&search_selected=true',
             'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB&sid=2d385a5c8721c18a0aca0b4388353818&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB%3Bsid%3D2d385a5c8721c18a0aca0b4388353818%3Bsb_price_type%3Dtotal%26%3B&ss=Bayonne%2C+Aquitaine%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=Bayonne&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1410844&dest_type=city&place_id_lat=43.49334&place_id_lon=-1.4751&search_pageview_id=f07e69615ba901c1&search_selected=true',
             'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB&sid=2d385a5c8721c18a0aca0b4388353818&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB%3Bsid%3D2d385a5c8721c18a0aca0b4388353818%3Bsb_price_type%3Dtotal%26%3B&ss=La+Rochelle%2C+Poitou-Charentes%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=La+Rochelle&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1438604&dest_type=city&iata=LRH&place_id_lat=46.15911&place_id_lon=-1.15204&search_pageview_id=f07e69615ba901c1&search_selected=true'
            ]

In [4]:
df_cities_url = pd.DataFrame({'city_name': cities, 'url_city': cities_url})
df_cities_url

,city_name,url_city
0,Mont-Saint-Michel,https://www.booking.com/searchresults.fr.html?...
1,St Malo,https://www.booking.com/searchresults.fr.html?...
2,Bayeux,https://www.booking.com/searchresults.fr.html?...
3,Le Havre,https://www.booking.com/searchresults.fr.html?...
4,Rouen,https://www.booking.com/searchresults.fr.html?...
5,Paris,https://www.booking.com/searchresults.fr.html?...
6,Amiens,https://www.booking.com/searchresults.fr.html?...
7,Lille,https://www.booking.com/searchresults.fr.html?...
8,Strasbourg,https://www.booking.com/searchresults.fr.html?...
9,Chateau du Haut Koenigsbourg,https://www.booking.com/searchresults.fr.html?...


# 2) Scrapping booking.com avec BeautifoulSoup

## Création du DataFrame contenant les 10 meilleurs hotels de chaque ville selon les recommandations de booking 

In [5]:
df_hotel_cols=['city_id', 'city_name', 'hotel_name', 'url', 'score', 'coord', 'description' ]
df_hotel=pd.DataFrame(columns=df_hotel_cols)
df_hotel

,city_id,city_name,hotel_name,url,score,coord,description


#### Test sur un hotel 

In [6]:
r = requests.get('https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB&sid=2d385a5c8721c18a0aca0b4388353818&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIDVgEaE2IAQGYAQ24ARfIAQzYAQPoAQH4AQKIAgGoAgO4AsKQw40GwAIB0gIkMzllYTE3YzYtYmJmNC00OWY1LWI0YTEtMGNhNGQ1MzkzMTRm2AIE4AIB%3Bsid%3D2d385a5c8721c18a0aca0b4388353818%3Bsb_price_type%3Dtotal%26%3B&ss=Paris%2C+%C3%8Ele-de-France%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=Paris&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1456928&dest_type=city&iata=PAR&place_id_lat=48.85668&place_id_lon=2.351476&search_pageview_id=f07e69615ba901c1&search_selected=true')
r

<Response [200]>

In [7]:
soup = BeautifulSoup(r.text, "html.parser")

In [8]:
#Récupérer l'url de l'hotel
url = soup.select('header.bui-spacer--medium a')[0].get('href')
print(url)
link = 'https://www.booking.com' + url
print("Page de l'hotel: ",link)
r2=requests.get(link)
print(r2)

/hotel/fr/hotelparismontparn.fr.html?sid=884da2f2ea81a3430027a83931bcf4b2
Page de l'hotel:  https://www.booking.com/hotel/fr/hotelparismontparn.fr.html?sid=884da2f2ea81a3430027a83931bcf4b2
<Response [200]>


In [9]:
soup2 = BeautifulSoup(r2.content, "html.parser")

#Description de l'hotel
print("Descption :",soup2.find("div",{"id":"property_description_content"}).text.replace('\n',' '), '\n')

#Nom de l'hotel
print("Nom: ",soup2.find('a','fn').get_text().split('\n')[1].split("\\n")[0], '\n')

#Latitude et Longitude
print("Coordonnées géographiques: ", soup2.find("a","jq_tooltip").get('data-atlas-latlng'), '\n')

#Note
print('Note: ', soup2.find("div","_9c5f726ff bd528f9ea6").get_text())

Descption :  Vous pouvez bénéficier d'une réduction Genius dans l'établissement Hotel Concorde Montparnasse ! Connectez-vous pour économiser. L'Hôtel Concorde Montparnasse se situe sur la rive gauche à Paris, dans le quartier de Montparnasse, à 20 minutes de route du stade Parc des Princes. Ses chambres élégantes comprennent la climatisation, une télévision par câble à écran plat, un plateau/bouilloire et une connexion Internet gratuite. Certaines chambres sont orientées à l'ouest et offrent une vue sur la tour Eiffel. Leur salle de bains privative est pourvue d'articles de toilette gratuits et d'un sèche-cheveux. Doté d'une terrasse ouverte quand le temps le permet, le restaurant ô'40 sert une cuisine maison moderne pour le déjeuner et le dîner, ainsi qu'un petit-déjeuner buffet. Au bar Le Wengé, vous pourrez aussi déguster une grande variété de boissons. De plus, l'Hôtel Concorde Montparnasse dispose de salles de réunion pouvant accueillir jusqu'à 150 participants. Lors de votre séjo

In [11]:
#Utilisation de try, except car certains hotels peuvent ne pas avoir de note car ajoutés récement sur le site. 
#Nous donnons la note de 7 à ces hotel

for i,j in tqdm(enumerate(df_cities_url.iloc[:,0])):
    r = requests.get(df_cities_url.iloc[:,1][i])
    soup = BeautifulSoup(r.content, "html.parser")
    for k,item in enumerate(soup.select('div.sr__card ')):
        url=item.select('header.bui-spacer--medium a')[0].get('href')
        lien = 'https://www.booking.com' + url
        r2=requests.get(lien)
        soup2 = BeautifulSoup(r2.content, "html.parser")
        try :
            df_hotel.loc[int(str(i)+str(k)), 'city_id']=i
            df_hotel.loc[int(str(i)+str(k)), 'city_name']=j
            df_hotel.loc[int(str(i)+str(k)),'description']=soup2.find("div",{"id":"property_description_content"}).text.replace('\n',' ')
            df_hotel.loc[int(str(i)+str(k)),'hotel_name']=soup2.find('a','fn').get_text().split('\n')[1].split("\\n")[0]
            df_hotel.loc[int(str(i)+str(k)),'coord']=soup2.find("a","jq_tooltip").get('data-atlas-latlng')
            df_hotel.loc[int(str(i)+str(k)),'url']=lien
            df_hotel.loc[int(str(i)+str(k)),'score']=soup2.find("div","_9c5f726ff bd528f9ea6").get_text()
        except:
            df_hotel.loc[int(str(i)+str(k)),'score']='7,0'

35it [15:48, 27.09s/it]


In [12]:
df_hotel

,city_id,city_name,hotel_name,url,score,coord,description
0,0,Mont-Saint-Michel,Le Relais Saint Michel,https://www.booking.com/hotel/fr/le-relais-sai...,"7,8","48.61758727,-1.51039615",Vous pouvez bénéficier d'une réduction Genius...
1,0,Mont-Saint-Michel,Les Terrasses Poulard,https://www.booking.com/hotel/fr/les-terrasses...,"7,3","48.63534943,-1.51037872",Vous pouvez bénéficier d'une réduction Genius...
2,0,Mont-Saint-Michel,Hotel De La Digue,https://www.booking.com/hotel/fr/de-la-digue.f...,"7,1","48.61688155,-1.51091784",L'hôtel De La Digue est un établissement trad...
3,0,Mont-Saint-Michel,Le Mouton Blanc,https://www.booking.com/hotel/fr/le-mouton-bla...,"7,2","48.63602298,-1.50989592",Vous pouvez bénéficier d'une réduction Genius...
4,0,Mont-Saint-Michel,La Mère Poulard,https://www.booking.com/hotel/fr/la-mere-poula...,"7,2","48.63508532,-1.51053965",Vous pouvez bénéficier d'une réduction Genius...
...,...,...,...,...,...,...,...
345,34,La Rochelle,ibis La Rochelle Vieux Port,https://www.booking.com/hotel/fr/ibis-la-roche...,"8,2","46.15582940,-1.14855800","Situé dans le quartier du Vieux-Port, dans le..."
346,34,La Rochelle,Maisons du Monde Hôtel & Suites - La Rochelle ...,https://www.booking.com/hotel/fr/inter-saint-j...,"8,4","46.15619304,-1.15443483",Vous pouvez bénéficier d'une réduction Genius...
347,34,La Rochelle,Central Park Hôtel & Spa,https://www.booking.com/hotel/fr/central-park-...,"8,7","46.15891333,-1.16082335",Vous pouvez bénéficier d'une réduction Genius...
348,34,La Rochelle,Kyriad La Rochelle Centre - Les Minimes,https://www.booking.com/hotel/fr/kyriad-la-roc...,"8,0","46.14630971,-1.15289390",Vous pouvez bénéficier d'une réduction Genius...


Séparation de latitude et longitude en deux variables

In [13]:
split_coord = df_hotel['coord'].str.split(',')
df_hotel.loc[:,'latitude'] = split_coord.apply(lambda x: x[0])
df_hotel.loc[:,'longitude'] = split_coord.apply(lambda x: x[1])
split_coord

0      [48.61758727, -1.51039615]
1      [48.63534943, -1.51037872]
2      [48.61688155, -1.51091784]
3      [48.63602298, -1.50989592]
4      [48.63508532, -1.51053965]
                  ...            
345    [46.15582940, -1.14855800]
346    [46.15619304, -1.15443483]
347    [46.15891333, -1.16082335]
348    [46.14630971, -1.15289390]
349    [46.15450177, -1.14467164]
Name: coord, Length: 350, dtype: object

In [14]:
#suprression de la colone coord qui n'est plus utile
df_hotel=df_hotel.drop('coord',1)

In [15]:
df_hotel.head()

,city_id,city_name,hotel_name,url,score,description,latitude,longitude
0,0,Mont-Saint-Michel,Le Relais Saint Michel,https://www.booking.com/hotel/fr/le-relais-sai...,"7,8",Vous pouvez bénéficier d'une réduction Genius...,48.61758727,-1.51039615
1,0,Mont-Saint-Michel,Les Terrasses Poulard,https://www.booking.com/hotel/fr/les-terrasses...,"7,3",Vous pouvez bénéficier d'une réduction Genius...,48.63534943,-1.51037872
2,0,Mont-Saint-Michel,Hotel De La Digue,https://www.booking.com/hotel/fr/de-la-digue.f...,"7,1",L'hôtel De La Digue est un établissement trad...,48.61688155,-1.51091784
3,0,Mont-Saint-Michel,Le Mouton Blanc,https://www.booking.com/hotel/fr/le-mouton-bla...,"7,2",Vous pouvez bénéficier d'une réduction Genius...,48.63602298,-1.50989592
4,0,Mont-Saint-Michel,La Mère Poulard,https://www.booking.com/hotel/fr/la-mere-poula...,"7,2",Vous pouvez bénéficier d'une réduction Genius...,48.63508532,-1.51053965


On transforme score, latitude et longitude de string en float pour les utiliser avec Plotly par la suite

In [16]:
df_hotel.dtypes

city_id        object
city_name      object
hotel_name     object
url            object
score          object
description    object
latitude       object
longitude      object
dtype: object

In [17]:
for i in range(len(df_hotel)):
    try :
        df_hotel['score'][i]=df_hotel['score'][i].replace(',', '.')
    except :
        pass

In [18]:
df_hotel[["latitude", "longitude", "score"]] = df_hotel[["latitude", "longitude", "score"]].apply(pd.to_numeric)

In [19]:
df_hotel.dtypes

city_id         object
city_name       object
hotel_name      object
url             object
score          float64
description     object
latitude       float64
longitude      float64
dtype: object

In [20]:
df_hotel['latitude'] = df_hotel['latitude'].astype(float)
df_hotel['longitude'] = df_hotel['longitude'].astype(float)

In [21]:
df_hotel['score'] = df_hotel['score'].astype(float)

On enregistre le DataFrame sous format csv

In [22]:
df_hotel.to_csv("top10_best_hotel_per_city.csv", index=False)